##  Importing libraries

In [ ]:
# We are getting specific categories from simplepbi
from simplepbi import token
from simplepbi import admin
from simplepbi import groups
from simplepbi import reports
from simplepbi import datasets
from simplepbi import imports
from simplepbi import utils
from simplepbi import azpause

## Parameters to start using the library

In [ ]:
TENANT_ID = '[Tenant ID]'
client_id = '[App or Client ID]'
client_secret = '[Secret Value]'

## Creating object token

In [ ]:
t = token.Token(TENANT_ID, client_id, None, None, client_secret, use_service_principal=True)
print(t)

## Creating API category objects

In [ ]:
a = admin.Admin(t.token)
g = groups.Groups(t.token)
d = datasets.Datasets(t.token)
r = reports.Reports(t.token)
im = imports.Imports(t.token)

## Testing requests

#### Groups API

In [ ]:
# get groups from group category

g.get_groups()

In [ ]:
# set a workspace id to work with

workspace_id = "[Workspace Id]"

In [ ]:
# add user to workspace blogging

g.add_user_group(workspace_id, groupUserAccessRight="Viewer", emailAddress="[Work Email]")

#### Admin API

In [ ]:
# get groups from admin category

a.get_groups(top=15, expand="reports")

In [ ]:
# get groups users type workspace filtered by active and not user specified

a.get_groups(top=50, expand="users", 
    filter="type eq 'Workspace' and state eq 'Active' and (not users/any(u: u/identifier eq '[Work Email]'))")

In [ ]:
# get groups expanding reports by report name

a.get_groups(top=50, expand="reports", filter="state eq 'Active' and (reports/any(u: u/name eq '[Report Name]'))")

In [ ]:
# Examples: https://powerbi.microsoft.com/en-us/blog/best-practices-to-prevent-getgroupsasadmin-api-timeout/

#### Datasets

In [ ]:
# get datasets from workspace blogging

d.get_datasets_in_group("[Workspace Id]")

In [ ]:
# get dataset blogdemos data sources

d.get_datasources_in_group("[Workspace Id]", '[Dataset Id]')

In [ ]:
# run a query against a dataset (semantic model)

d.execute_queries_in_group("[Workspace Id]", '[Dataset Id]', 
                           "EVALUATE [Table Name]", return_pandas=True )

#### Amazing requests

In [ ]:
# Get all actifacts a usar has access

res = a.get_user_artifact_access_preview("[User graph Id]", return_pandas=False)

utils.to_pandas(res,"ArtifactAccessEntities")

In [ ]:
# Get unused artifacts from a workspace

res = a.get_unused_artifacts("[Workspace Id]")
res

In [ ]:
# List of Power BI reports that are shared with the whole organization through links.

a.get_widely_shared_artifacts_links_shared_to_whole_organization()


In [ ]:
# Set report and destinations for migration between workspaces
# workspace_id:  It's Blogging workspace

report_id = "[Report Id]" #Transporte Cba report

workspace_destination ="[Workspace Id]" #Event Session workspace

In [ ]:
# Copy report between workspaces (migration)

r.simple_copy_reports_between_groups(workspace_id, report_id, workspace_destination)

In [ ]:
# Get orphan dataflows (dataflows that are not linked to dataset)

a.get_orphan_dataflows_preview()

In [ ]:
# Get dataflow to know more about it where is it (workspace)

a.get_dataflows(filter="objectId eq [Dataflow Id]")

In [ ]:
# Scanner API to get important info
# Get modified workspaces

workspaces_list = a.get_modified_workspaces_preview(excludePersonalWorkspaces=True, modifiedSince=None)
print(workspaces_list)

In [ ]:
# Initiates a call to receive metadata for the requested list of workspaces
# 100 workspaces limit

a.post_workspace_info(workspaces_list[0], lineage=True, 
                      datasourceDetails=True, datasetSchema=True, 
                      datasetExpressions=True, getArtifactUsers=True)


In [ ]:
# Check the status of the scanning with a scan id

a.get_scan_status_preview('[Scan Id]')

In [ ]:
# Get scan result

scan_result = a.get_scan_result_preview('[Scan Id]')

In [ ]:
# Convert scan result of a single artifact to pandas
# scan_result

df_scan = utils.get_artifact_from_scan_preview(scan_result, artifact="datasets")
df_scan.head()

#### Other helpfull APIs

In [ ]:
# Set Azure DevOps Repo variables

organization = '[Organization name]'
project = '[Project name]'
repository_id = '[Repo Name]'
path = f'[Path]'
devopsKey = '[Personal access token]'

In [ ]:
# Import pbix files from Azure DevOps Repo

im.simple_import_from_devops(organization, project, repository_id, path, devopsKey, "[Workspace Id]")

In [ ]:
# Create token for Azure resource manager (different from Previous one)

azure = azpause.Azpause(TENANT_ID, client_id, client_secret)

In [ ]:
# Resume Fabric capacity

azure.resume_resource("[Suscription Id]", "[Resource group name]", "FABRIC", "[Resource name]")

In [ ]:
# Pause Fabric capacity

azure.pause_resource("[Suscription Id]", "[Resource group name]", "FABRIC", "[Resource name]")